In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install torch torchvision torchaudio
!pip install torch --upgrade
!pip install triton
!pip install nnunetv2

# Prepare the data for the nnunetV2

### The following cell only if the mapping is different or the labels have decimals (e.g: 1.00002 instead of 1)
### This sometime happen in generated data or data with noise so we standardize the labels mapping

In [ ]:
# === Imports ===
import os
import nibabel as nib
import numpy as np

# === User Input Required: Define Paths ===
# Please update these paths to your dataset directories

# Input directory containing NIfTI label files
input_dir = "/path/to/input/labels"  # e.g., "./cycle_generated_nifti/labels"

# Output directory to save processed labels
output_dir = "/path/to/output/processed_labels"  # e.g., "./processed_labels"

# Create output directory if it does not exist
os.makedirs(output_dir, exist_ok=True)

# === Define mapping dictionary ===
# Maps original intensity values to desired class labels

value_map = {
    #replace the first columns with your actual labels if they are not standardized to (0,1,2,3,4)
    0: 0,
    20: 1,
    60: 2,
    100: 3,
    140: 4
}

# === Process each label file ===
for file in os.listdir(input_dir):
    if file.endswith(".nii"):
        file_path = os.path.join(input_dir, file)
        
        # Load NIfTI file
        nifti_img = nib.load(file_path)
        data = nifti_img.get_fdata()
        
        # Initialize mapped data with zeros (int32 type)
        mapped_data = np.zeros_like(data, dtype=np.int32)
        
        # Tolerance for floating point matching (in intensity units)
        tolerance = 5
        
        for old_val, new_val in value_map.items():
            mask = np.abs(data - old_val) < tolerance
            mapped_data[mask] = new_val
        
        # Create new NIfTI image with mapped integer classes
        new_nifti = nib.Nifti1Image(mapped_data, affine=nifti_img.affine, header=nifti_img.header)
        
        # Ensure data is saved as integers in NIfTI
        new_nifti.set_data_dtype(np.int32)
        
        # Save to output directory with the same name
        output_path = os.path.join(output_dir, file)
        nib.save(new_nifti, output_path)
        
        print(f"Processed and saved {file}")

print(f"✅ All labels processed with correct integer classes and saved in {output_dir}")


### The following cells are the mandatory nnunetV2 preparation with the required structure (must be run)

In [ ]:
# === Imports ===
import os

# === User Input Required: Define Paths ===
# Please update these paths to your dataset directories

# Root directory for nnU-Net raw dataset
nnunet_raw = "/path/to/nnUNet_raw/DatasetXXX_YOURDATASET"  # e.g., "./nnUNet_raw/Dataset033_EMIDEC"
os.makedirs(nnunet_raw, exist_ok=True)

# Original dataset directories
image_train_dir = "/path/to/original/images_train"  # e.g., "./nifti/images_train"
image_test_dir = "/path/to/original/images_test"    # e.g., "./nifti/images_test"
mask_train_dir = "/path/to/original/masks_train"    # e.g., "./nifti/masks_train"
mask_test_dir = "/path/to/original/masks_test"      # e.g., "./nifti/masks_test"

# Generated dataset directories (e.g., after CycleGAN or preprocessing)
generated_image_train_dir = "/path/to/generated/images"  # e.g., "./cycle_generated_nifti/images"
generated_mask_train_dir = "/path/to/generated/masks"    # e.g., "./processed_labels"

# === Define nnU-Net subdirectories ===
nnunet_imagesTr = os.path.join(nnunet_raw, "imagesTr")
nnunet_labelsTr = os.path.join(nnunet_raw, "labelsTr")
nnunet_imagesTs = os.path.join(nnunet_raw, "imagesTs")

# === Create required directories ===
os.makedirs(nnunet_imagesTr, exist_ok=True)
os.makedirs(nnunet_labelsTr, exist_ok=True)
os.makedirs(nnunet_imagesTs, exist_ok=True)

print("✅ nnU-Net dataset directories created successfully!")


In [ ]:
# === Imports ===
import shutil
import glob
import os

# === User Input Required: Define Paths ===
# Please ensure these variables are defined prior to running this script:
# - image_train_dir
# - mask_train_dir
# - generated_image_train_dir
# - generated_mask_train_dir
# - image_test_dir
# - nnunet_imagesTr
# - nnunet_labelsTr
# - nnunet_imagesTs

# Example:
# image_train_dir = "/path/to/images_train"
# mask_train_dir = "/path/to/masks_train"
# generated_image_train_dir = "/path/to/generated/images"
# generated_mask_train_dir = "/path/to/generated/masks"
# image_test_dir = "/path/to/images_test"
# nnunet_imagesTr = "/path/to/nnUNet_raw/DatasetXXX/imagesTr"
# nnunet_labelsTr = "/path/to/nnUNet_raw/DatasetXXX/labelsTr"
# nnunet_imagesTs = "/path/to/nnUNet_raw/DatasetXXX/imagesTs"

# === Function to copy and rename files as .nii.gz ===
def copy_and_rename(src_dir, dest_dir):
    for file in glob.glob(os.path.join(src_dir, "*.nii")):
        case_name = os.path.basename(file).replace(".nii", ".nii.gz")
        shutil.copy(file, os.path.join(dest_dir, case_name))

# === Copy original training images ===
copy_and_rename(image_train_dir, nnunet_imagesTr)

# === Copy original training masks ===
copy_and_rename(mask_train_dir, nnunet_labelsTr)

# === Copy generated training images ===
copy_and_rename(generated_image_train_dir, nnunet_imagesTr)

# === Copy generated training masks ===
copy_and_rename(generated_mask_train_dir, nnunet_labelsTr)

# === Copy test images (no labels) ===
copy_and_rename(image_test_dir, nnunet_imagesTs)

print("✅ All data (original + generated) moved and renamed successfully!")


In [ ]:
import json

dataset_json = {
    "name": "EMIDEC",
    "channel_names": {
        "0": "CMR"  # You can change this name to match your data modality
    },
    "description": "Cardiac MRI dataset for nnU-Net",
    "tensorImageSize": "3D",
    "reference": "EMIDEC Challenge",
    "licence": "CC-BY-SA 4.0",
    "release": "1.0",
    "modality": {"0": "CMR"},
    "labels": {
        "background": 0,
        "Cavity": 1,
        "Normal Myocardium": 2,
        "Infarction": 3,
        "No-Reflow": 4
    },
    "numTraining": len(os.listdir(nnunet_imagesTr)),
    "file_ending": ".nii.gz"
}

# Save JSON file
json_path = os.path.join(nnunet_raw, "dataset.json")
with open(json_path, "w") as f:
    json.dump(dataset_json, f, indent=4)

print("Dataset JSON created successfully!")

In [ ]:
print("Training images:", len(os.listdir(nnunet_imagesTr)))
print("Training masks:", len(os.listdir(nnunet_labelsTr)))
print("Test images:", len(os.listdir(nnunet_imagesTs)))


In [ ]:
import os

# Set environment variables for nnU-Net
os.environ['nnUNet_raw'] = '/kaggle/working/nnUNet_raw'
os.environ['nnUNet_preprocessed'] = '/kaggle/working/nnUNet_preprocessed'
os.environ['nnUNet_results'] = '/kaggle/working/nnUNet_results'

# Verify if the environment variables are set correctly
print("nnUNet_raw:", os.environ['nnUNet_raw'])
print("nnUNet_preprocessed:", os.environ['nnUNet_preprocessed'])
print("nnUNet_results:", os.environ['nnUNet_results'])


In [ ]:
# === Imports ===
import os

# === Function to rename files in a given directory ===
def rename_files(data_dir):
    # List all files in the directory
    files = os.listdir(data_dir)

    # Group files by case identifier (e.g., P036, P056, etc.)
    case_dict = {}
    for file in files:
        case_id = file.split('.')[0]  # Extract the case identifier (e.g., 'P036' from 'P036.nii.gz')
        if case_id not in case_dict:
            case_dict[case_id] = []
        case_dict[case_id].append(file)

    # Rename the files in each group
    for case_id, case_files in case_dict.items():
        for i, file in enumerate(sorted(case_files)):  # Sort files to ensure correct order
            new_name = f"{case_id}_{i:04d}.nii.gz"  # Format: 'Case_0000.nii.gz'
            old_path = os.path.join(data_dir, file)
            new_path = os.path.join(data_dir, new_name)
            os.rename(old_path, new_path)
            print(f"Renamed {file} to {new_name}")

# === User Input Required: Define Directories ===
# Please update these paths to your dataset directories

images_dir = '/path/to/nnUNet_raw/DatasetXXX/imagesTr'   # e.g., "./nnUNet_raw/Dataset033_EMIDEC/imagesTr"
labels_dir = '/path/to/nnUNet_raw/DatasetXXX/labelsTr'   # e.g., "./nnUNet_raw/Dataset033_EMIDEC/labelsTr"

# === Rename files in both the images and labels directories ===
rename_files(images_dir)
# rename_files(labels_dir)  # Uncomment if you want to rename labels as well

print("✅ File renaming completed.")


In [ ]:
import os

# Set environment variables for nnU-Net
os.environ['nnUNet_raw'] = '/kaggle/working/nnUNet_raw'
os.environ['nnUNet_preprocessed'] = '/kaggle/working/nnUNet_preprocessed'
os.environ['nnUNet_results'] = '/kaggle/working/nnUNet_results'

# Verify if the environment variables are set correctly
print("nnUNet_raw:", os.environ['nnUNet_raw'])
print("nnUNet_preprocessed:", os.environ['nnUNet_preprocessed'])
print("nnUNet_results:", os.environ['nnUNet_results'])


In [ ]:
!nnUNetv2_plan_and_preprocess -d 033 --verify_dataset_integrity


# Train with the defualt loss and trainer
### To apply a custom loss and trainer skip this part

In [ ]:
!nnUNetv2_train 033 2d 0  #replace 033 with your actual dataset number


### To run inference you need to rename the testing images too to case_0000

In [ ]:
def rename_files(data_dir):
    # List all the files in the directory
    files = os.listdir(data_dir)

    # Group files by case identifier (e.g., P036, P056, etc.)
    case_dict = {}
    for file in files:
        case_id = file.split('.')[0]  # Get the case identifier (e.g., 'P036' from 'P036.nii.gz')
        if case_id not in case_dict:
            case_dict[case_id] = []
        case_dict[case_id].append(file)

    # Now rename the files in each group
    for case_id, case_files in case_dict.items():
        for i, file in enumerate(sorted(case_files)):  # Sorting the files by name ensures correct order
            new_name = f"{case_id}_{i:04d}.nii.gz"  # Format the new name as 'Case_0000'
            old_path = os.path.join(data_dir, file)
            new_path = os.path.join(data_dir, new_name)
            os.rename(old_path, new_path)
            print(f"Renamed {file} to {new_name}")

test_dir = '/path/to/nnUNet_raw/DatasetXXX/imagesTs'

# Rename files in both the images and labels directories
rename_files(test_dir)

In [ ]:
import os
import json

# Define the base directory
base_dir = "/path/to/nnUNet_results/Dataset033_EMIDEC/nnUNetTrainer__nnUNetPlans__2d/"

# Create the directory if it doesn't exist
os.makedirs(base_dir, exist_ok=True)

# Define the plan.json content
plan_json = {
    "dataset_name": "Dataset033_EMIDEC",
    "plans_name": "nnUNetPlans",
    "original_median_spacing_after_transp": [1.0, 1.0, 1.0],
    "original_median_shape_after_transp": [7, 128, 128],
    "image_reader_writer": "SimpleITKIO",
    "transpose_forward": [0, 1, 2],
    "transpose_backward": [0, 1, 2],
    "configurations": {
        "2d": {
            "data_identifier": "nnUNetPlans_2d",
            "preprocessor_name": "DefaultPreprocessor",
            "batch_size": 30,
            "patch_size": [128, 128],
            "median_image_size_in_voxels": [128.0, 128.0],
            "spacing": [1.0, 1.0],
            "normalization_schemes": ["ZScoreNormalization"],
            "use_mask_for_norm": [False],
            "resampling_fn_data": "resample_data_or_seg_to_shape",
            "resampling_fn_seg": "resample_data_or_seg_to_shape",
            "resampling_fn_data_kwargs": {"is_seg": False, "order": 3, "order_z": 0, "force_separate_z": None},
            "resampling_fn_seg_kwargs": {"is_seg": True, "order": 1, "order_z": 0, "force_separate_z": None},
            "resampling_fn_probabilities": "resample_data_or_seg_to_shape",
            "resampling_fn_probabilities_kwargs": {"is_seg": False, "order": 1, "order_z": 0, "force_separate_z": None},
            "architecture": {
                "network_class_name": "dynamic_network_architectures.architectures.unet.PlainConvUNet",
                "arch_kwargs": {
                    "n_stages": 6,
                    "features_per_stage": [32, 64, 128, 256, 512, 512],
                    "conv_op": "torch.nn.modules.conv.Conv2d",
                    "kernel_sizes": [[3, 3]] * 6,
                    "strides": [[1, 1], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2]],
                    "n_conv_per_stage": [2] * 6,
                    "n_conv_per_stage_decoder": [2] * 5,
                    "conv_bias": True,
                    "norm_op": "torch.nn.modules.instancenorm.InstanceNorm2d",
                    "norm_op_kwargs": {"eps": 1e-05, "affine": True},
                    "dropout_op": None,
                    "dropout_op_kwargs": None,
                    "nonlin": "torch.nn.LeakyReLU",
                    "nonlin_kwargs": {"inplace": True},
                },
                "_kw_requires_import": ["conv_op", "norm_op", "dropout_op", "nonlin"],
            },
            "batch_dice": True,
        }
    },
    "experiment_planner_used": "ExperimentPlanner",
    "label_manager": "LabelManager",
    "foreground_intensity_properties_per_channel": {
        "0": {
            "max": 3093.0,
            "mean": 2396.349853515625,
            "median": 2377.0,
            "min": 1806.0,
            "percentile_00_5": 1882.0,
            "percentile_99_5": 3093.0,
            "std": 244.4371795654297,
        }
    },
}

# Define the dataset.json content
dataset_json = {
    "name": "EMIDEC",
    "channel_names": {"0": "CMR"},
    "description": "Cardiac MRI dataset for nnU-Net",
    "tensorImageSize": "3D",
    "reference": "EMIDEC Challenge",
    "licence": "CC-BY-SA 4.0",
    "release": "1.0",
    "modality": {"0": "CMR"},
    "labels": {"background": 0, "Cavity": 1, "Normal Myocardium": 2, "Infarction": 3, "No-Reflow": 4},
    "numTraining": 85,
    "file_ending": ".nii.gz",
}

# Write the files
with open(os.path.join(base_dir, "plans.json"), "w") as f:
    json.dump(plan_json, f, indent=4)

with open(os.path.join(base_dir, "dataset.json"), "w") as f:
    json.dump(dataset_json, f, indent=4)

print("✅ Directory and required files created successfully!")


In [ ]:
# === nnU-Net v2 Inference Command ===
# Please update the paths below to your own directories and checkpoint

!nnUNetv2_predict \
    -i /path/to/nnUNet_raw/DatasetXXX/imagesTs \  # Path to imagesTs directory inside your nnUNet dataset
    -o /path/to/nnUNet_predictions/test_predictions \  # Output directory for predictions
    -d XXX \  # Dataset number (e.g., 033)
    -c 2d \  # Trainer configuration (e.g., '2d' or '3d_fullres')
    -f 0 \  # Fold number used for inference (e.g., 0)
    -chk /path/to/model_checkpoint/checkpoint_latest.pth  # Path to model checkpoint (.pth file)


# Train with custom loss and trainer (example distance weighted but the same method applied for any other custom loss or trainer)

In [ ]:
import os

# Define the new loss file path
# === User Input Required: Define Custom Loss File Path ===
# Please replace this path with the actual path to your nnU-Net v2 library installation
# For example, in Colab or your local environment, run:
# import nnunetv2; print(nnunetv2.__file__) to locate the base package path
custom_loss_path = "/path/to/nnunetv2/training/nnUNetTrainer/variants/loss/CustomDistanceLoss.py"

custom_loss_code = """import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomDistanceLoss(nn.Module):
    def __init__(self, class_weights=None):
        super(CustomDistanceLoss, self).__init__()
        self.class_weights = torch.tensor(class_weights, dtype=torch.float32) if class_weights else None

    def forward(self, prediction, target):
        # Ensure we have lists of tensors for multi-scale outputs
        if not isinstance(prediction, list):
            prediction = [prediction]
        if not isinstance(target, list):
            target = [target]

        num_classes = prediction[0].shape[1]
        total_loss = 0.0
        smooth = 1e-6

        for i, (pred, tgt) in enumerate(zip(prediction, target)):
            pred = pred.to(dtype=torch.float32, device=tgt.device)
            tgt = tgt.to(dtype=torch.long, device=pred.device)

            # Convert target to one-hot
            targets_onehot = F.one_hot(tgt.squeeze(1), num_classes).permute(0, 3, 1, 2).float()

            # ----- DISTANCE LOSS -----
            # Compute squared difference between prediction and target
            squared_difference = (pred - targets_onehot) ** 2
            distance_loss = torch.mean(squared_difference, dim=(2, 3))

            if self.class_weights is not None:
                self.class_weights = self.class_weights.to(pred.device)  # Move to the same device
                distance_loss = distance_loss * self.class_weights.view(1, num_classes)  # Apply weights
            distance_loss = distance_loss.mean()

            total_loss += distance_loss

        return total_loss / len(prediction)
"""

# Write the loss code to the file
with open(custom_loss_path, "w") as f:
    f.write(custom_loss_code)

print("✅ CustomDistanceLoss.py has been created successfully!")

# Verify
with open(custom_loss_path, "r") as f:
    print(f.read())


In [ ]:
# === User Input Required: Define Trainer File Path ===
# Please replace this path with the actual path to your nnUNet v2 library installation.
# You can find it by running:
# import nnunetv2
# print(nnunetv2.__file__)

trainer_path = "/path/to/nnunetv2/training/nnUNetTrainer/variants/nnUNetTrainer_CustomDistance.py"

trainer_code = """import torch
from nnunetv2.training.nnUNetTrainer.nnUNetTrainer import nnUNetTrainer
from nnunetv2.training.nnUNetTrainer.variants.loss.CustomDistanceLoss import CustomDistanceLoss

class nnUNetTrainer_CustomDistance(nnUNetTrainer):
    def _build_loss(self):
        # Define class weights for loss function
        class_weights = [0.0020701, 0.01770402, 0.0211376, 0.12820969, 1.0]
        # class_weights = [0.1, 0.2, 0.3, 1.2820969, 1.5]
        return CustomDistanceLoss(class_weights=class_weights)
"""

# Write the trainer code to the file
with open(trainer_path, "w") as f:
    f.write(trainer_code)

print("✅ nnUNetTrainer_CustomDistance.py has been created successfully!")

# Verify
with open(trainer_path, "r") as f:
    print(f.read())


In [ ]:
!nnUNetv2_train 033 2d 0 -tr nnUNetTrainer_CustomDistance  #replace 033 with your actual dataset number


### To run inference using the custom trainer do this

In [ ]:
test_dir = '/path/to/nnUNet_raw/DatasetXXX/imagesTs'

# Rename files in both the images and labels directories
rename_files(test_dir)

In [ ]:
# === nnU-Net v2 Inference Command with Custom Trainer ===
# Please update the paths below to match your environment and dataset.

!nnUNetv2_predict \
    -i /path/to/nnUNet_raw/DatasetXXX/imagesTs \  # Path to imagesTs directory inside your nnUNet dataset
    -o /path/to/nnUNet_predictions \  # Output directory for predictions
    -d XXX \  # Dataset number (e.g., 033)
    -c 2d \  # Trainer configuration (e.g., '2d' or '3d_fullres')
    -f 0 \  # Fold number used for inference (e.g., 0)
    -tr nnUNetTrainer_CustomDistance \  # Name of your custom trainer class
    -chk /path/to/model_checkpoint/checkpoint_best.pth  # Path to model checkpoint (.pth file)


# Visualise Predictions

In [ ]:
# === Imports ===
import os

# === User Input Required: Define Data Directory ===
# Please update this path to your imagesTs directory

data_dir = '/path/to/nnUNet_raw/DatasetXXX/imagesTs'  # e.g., "./nnUNet_raw/Dataset033_EMIDEC/imagesTs"

# === Rename .nii.gz files back to .nii ===
for file in os.listdir(data_dir):
    if file.endswith(".nii.gz"):
        old_path = os.path.join(data_dir, file)
        new_name = file.replace(".nii.gz", ".nii")
        new_path = os.path.join(data_dir, new_name)
        os.rename(old_path, new_path)
        print(f"✅ Renamed {file} back to {new_name}")

print("🎯 All files renamed back to .nii")


In [ ]:
# === Imports ===
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

# === User Input Required: Define Instance ID and Paths ===
instance_file = "YOUR_INSTANCE_ID"  # e.g., "P050"

# Define file paths below based on your directory structure

input_img_path = f"/path/to/nnUNet_raw/DatasetXXX/imagesTs/{instance_file}_0000.nii"  # e.g., "./nnUNet_raw/Dataset033_EMIDEC/imagesTs/P050_0000.nii"
gt_path = f"/path/to/ground_truth/masks_test/{instance_file}.nii"  # e.g., "./nifti/masks_test/P050.nii"
pred_path = f"/path/to/nnUNet_predictions/test_predictions/{instance_file}.nii.gz"  # e.g., "./nnUNet_predictions/test_predictions/P050.nii.gz"

# === Load data ===
input_img = nib.load(input_img_path).get_fdata()
gt_mask = nib.load(gt_path).get_fdata()
pred_mask = nib.load(pred_path).get_fdata()

# === Function to get middle slice if 3D ===
def get_middle_slice(arr):
    if arr.ndim == 3:
        return arr[:, :, arr.shape[2] // 2]
    return arr

input_slice = get_middle_slice(input_img)
gt_slice = get_middle_slice(gt_mask)
pred_slice = get_middle_slice(pred_mask)

# === Plotting ===
plt.figure(figsize=(15,5))

plt.subplot(1,3,1)
plt.imshow(input_slice, cmap='gray')
plt.title("Input Image")
plt.axis("off")

plt.subplot(1,3,2)
plt.imshow(gt_slice, cmap='jet')
plt.title("Ground Truth Mask")
plt.axis("off")

plt.subplot(1,3,3)
plt.imshow(pred_slice, cmap='jet')
plt.title("Predicted Mask")
plt.axis("off")

plt.suptitle(f"{instance_file} - Input | Ground Truth | Prediction")
plt.show()
